<a href="https://colab.research.google.com/github/tmoura/machinelearning/blob/master/Series_Temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

path = 'https://raw.githubusercontent.com/EraylsonGaldino/dataset_time_series/master/goldman.txt'
dados = pd.read_csv(path)

In [3]:
min = 0
max = 1
scaler = MinMaxScaler(feature_range=(min, max)).fit(dados)
serie_normalizada = scaler.transform(dados.values.reshape(1, -1))

dados['Normalizado'] = serie_normalizada[0]

In [4]:
def create_sliding_windows(series, window_size):
   
    list_of_sliding_windows = []
    list_size_to_iterate = len(series) - window_size 
    for i in range(0, list_size_to_iterate):
        window = series[i: i + window_size + 1]
        list_of_sliding_windows.append(window)

    return np.array(list_of_sliding_windows).reshape(len(list_of_sliding_windows), window_size+1)

## **DÚVIDA NA PRÓXIMA CÉLULA**

Perguntas:

1) No código que você me passou, você divide a base de treinamento e teste por intervalo de datas, mas no paper você simplesmente separou os 80% primeiros padrões para treinamento e os últimos 20% para teste.

Aqui, depois da normalização, eu vou criar as janelas com os lags de 20 para depois separar em treinamento e teste, confere?

In [20]:
windows_train = create_sliding_windows(dados['Normalizado'], 20)

In [ ]:
perc_val = 0.2
tam_val = np.fix(len(windows_train) * perc_val).astype(int)

X_train, y_train = windows_train[0:-tam_val, 0:-1], windows_train[0:-tam_val, -1]
X_val, y_val = windows_train[-tam_val:, 0:-1], windows_train[-tam_val:, -1]

In [ ]:
pontos_anteriores_test = train_data.values[-20:]
test = np.hstack([pontos_anteriores_test, test_data.values])

windows_test = create_sliding_windows(test, 20)
X_test, y_test = windows_test[:, 0:-1], windows_test[:, -1]

In [30]:
def select_lag_acf(serie, max_lag):
    from statsmodels.tsa.stattools import acf
    x = serie[0: max_lag+1]
    
    acf_x, confint = acf(serie, nlags=max_lag, alpha=.05, fft=False,
                             unbiased=False)
    
    
    limiar_superior = confint[:, 1] - acf_x
    limiar_inferior = confint[:, 0] - acf_x

    lags_selecionados = []
    
    for i in range(1, max_lag+1):

        
        if acf_x[i] >= limiar_superior[i] or acf_x[i] <= limiar_inferior[i]:
            lags_selecionados.append(i-1)  #-1 por conta que o lag 1 em python é o 0
    
    #caso nenhum lag seja selecionado, essa atividade de seleção para o gridsearch encontrar a melhor combinação de lags
    if len(lags_selecionados)==0:


        print('NENHUM LAG POR ACF')
        lags_selecionados = [i for i in range(max_lag)]

    print('LAGS', lags_selecionados)

    #inverte o valor dos lags para usar na lista de dados
    lags_selecionados = [max_lag - (i+1) for i in lags_selecionados]

    return lags_selecionados

## **DÚVIDA NA PRÓXIMA CÉLULA**

Perguntas:

1) Fiz testes com a base goldman, star e msft e praticamente os lags selecionados foram quase os mesmos, com a remoção de um ou outro lag, praticamente as 3 bases usaram todos os lags. É isso mesmo?? O código da função acima ("select_lag_acf") tá correto??

Resolvendo isso, eu transformo as bases, já as deixo prontas em python, salvo em CSV e só rodo meu código no MATLAB e pronto!

Tive ideia de outro artigo para a gente trabalhar!!!

In [ ]:
lags_sel = select_lag_acf(dados['Normalizado'], 20)

print(lags_sel)